**Install Required Packages**

In [47]:
!pip install langchain langchain-cohere PyMuPDF sentence-transformers faiss-cpu pypdf


In [48]:
!pip install pypdf


**Imports**

In [49]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_cohere import ChatCohere
import pickle


**Set your Cohere API key**

In [ ]:
# Enter your Cohere API key interactively (will not be stored in the notebook)
from getpass import getpass
key = getpass('Enter Cohere API key: ')
import os
if key:
    os.environ['COHERE_API_KEY'] = key
else:
    print('No key entered; make sure COHERE_API_KEY is set in the environment or use the Streamlit sidebar to provide it.')

**Load Indian Constitution PDF**

In [51]:
pdf_path = "/content/constitution_of_india.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

**Chunking (by Article/Section size)**

In [52]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"✅ Total chunks created: {len(chunks)}")

✅ Total chunks created: 837


**Create Embeddings & Vector DB**


In [53]:
# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_db = FAISS.from_documents(chunks, embedding_model)

print("✅ Vector DB created successfully")


✅ Vector DB created successfully


**Save Vector DB as Pickle**

In [54]:
# Save vector DB for later use (for Streamlit)
with open("vector_db.pkl", "wb") as f:
    pickle.dump(vector_db, f)

print("✅ Vector DB saved as vector_db.pkl")


✅ Vector DB saved as vector_db.pkl


**Load Vector DB from Pickle**

In [55]:
# Load saved vector DB (useful for Streamlit)
with open("vector_db.pkl", "rb") as f:
    vector_db = pickle.load(f)

print("✅ Vector DB loaded successfully")


✅ Vector DB loaded successfully


**Setup LLM & RAG Chain**

In [56]:
# Initialize Cohere LLM (use supported model)
llm = ChatCohere(model="command-xlarge-nightly")

In [57]:
# Setup retriever
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [58]:
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [59]:
print("✅ RAG QA chain ready")

✅ RAG QA chain ready


**Ask Questions**

In [60]:
query = "What is Article 370 in the Indian Constitution?"
result = qa_chain({"query": query})

# Display answer
print("\n💡 Answer:\n", result["result"])
print("\n📌 Sources (Articles/Sections Used):")
for doc in result["source_documents"]:
    print(doc.page_content[:300], "...\n")



💡 Answer:
 Article 370 of the Indian Constitution is a temporary provision that granted special autonomous status to the state of Jammu and Kashmir. It was included in the Constitution under Part XXI, which deals with Temporary, Transitional, and Special Provisions. Here are the key points of Article 370:

1. **Special Status for Jammu and Kashmir**: Article 370 provided that the provisions of the Indian Constitution, except those related to defense, foreign affairs, finance, and communications, would not automatically apply to Jammu and Kashmir. The state had its own constitution and could make laws on matters not explicitly ceded to the central government.

2. **President's Power to Extend Provisions**: The President of India had the authority to extend other provisions of the Constitution to Jammu and Kashmir, but only with the concurrence of the state's government.

3. **Instrument of Accession**: The powers of the Parliament to make laws for Jammu and Kashmir were limited to thos

In [61]:
query = "how to deal with failurein  Indian Constitution?"
result = qa_chain({"query": query})

In [62]:
print("\n💡 Answer:\n", result["result"])
print("\n📌 Sources (Articles/Sections Used):")
for doc in result["source_documents"]:
    print(doc.page_content[:300], "...\n")


💡 Answer:
 The Indian Constitution does not explicitly outline a specific process for dealing with "failure" in the sense of personal or individual setbacks. However, it provides a framework for addressing systemic failures, governance issues, and constitutional challenges through various mechanisms. Here’s how the Constitution deals with such situations:

1. **Judicial Review (Article 13, 32, 136, 226, 227):**  
   - The Supreme Court and High Courts have the power to review laws and actions of the government to ensure they comply with the Constitution. If a law or action violates fundamental rights or constitutional principles, it can be struck down.

2. **Fundamental Rights (Part III, Articles 12-35):**  
   - Citizens can seek redressal for violations of their fundamental rights through writs (habeas corpus, mandamus, prohibition, quo warranto, certiorari) under Article 32 (Supreme Court) and Article 226 (High Courts).

3. **Parliamentary Oversight:**  
   - Parliament exercises o

In [63]:
query = "hello"
result = qa_chain({"query": query})

In [64]:
print("\n💡 Answer:\n", result["result"])
print("\n📌 Sources (Articles/Sections Used):")
for doc in result["source_documents"]:
    print(doc.page_content[:300], "...\n")


💡 Answer:
 Hello! How can I assist you today?

📌 Sources (Articles/Sections Used):
to him immediately before such commencement.
378. (1) The members of the Public Service
Commission for the Dominion of India holding office
immediately before the commencement of this
Constitution shall, unless they have elected otherwise,
become on such commencement the members of the
Public Servic ...

(2) The President may send messages to either House
of Parliament, whether with respect to a Bill then pending
in Parliament or otherwise, and a House to which any
message is so sent shall with all convenient despatch
consider any matter required by the message to be taken
into consideration.
87. (1 ...

2*** of a State or such person as he
may direct in the case of services and posts in connection
with the affairs of the  State, to make  rules regulating the
recruitment, and the conditions of service of persons
appointed, to such services and posts until provision in
that behalf is made by or under ...

In [ ]:
# Interactive test using the new backend module
from backend.rag_bot import get_default_bot

# Initialize bot (assumes vector_db.pkl in notebook folder). If you need to pass your API key, set COHERE_API_KEY in env or pass cohere_api_key=...
bot = get_default_bot(vector_db_path="vector_db.pkl")

# Simple input loop: run in notebook to ask a single question
query = input('Enter question about the Indian Constitution: ')
if query.strip():
    resp = bot.answer(query)
    print('
Answer:
', resp.get('answer'))
    print('
Sources:')
    for i, s in enumerate(resp.get('sources', []), 1):
        print(f'--- Source {i} ---')
        if s.get('metadata'):
            print('Metadata:', s['metadata'])
        print(s.get('page_content', '')[:1000])
else:
    print('No question entered.')
